In [1]:
import pandas as pd
import numpy as np

In [2]:
RANDOM_STATE = 42

results_regression = pd.DataFrame(columns = ['model', 'task', 'R2'])
results_classification = pd.DataFrame(columns = ['model', 'task', 'f1', 'accuracy'])

https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

In [3]:
data = pd.read_csv('boston.csv')
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


1. Разделите выборку на обучающую и тестовую в отношении 80%/20%, предварительно выделив целевую переменную (колонка 'MEDV').

In [4]:
from sklearn.model_selection import train_test_split

# Загрузка данных
data = pd.read_csv('boston.csv')

# Выделение целевой переменной и признаков
X = data.drop(columns=['MEDV'])
y = data['MEDV']

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)

# Проверка размеров полученных выборок
print("Размер обучающей выборки:", X_train.shape)
print("Размер тестовой выборки:", X_test.shape)



Размер обучающей выборки: (404, 13)
Размер тестовой выборки: (102, 13)


2. Обучите стандартную регрессию, а также Ridge и  Lasso с параметрами по умолчанию и выведите их R2 на тестовой выборке

In [5]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import r2_score

# Создание и обучение моделей
lr_model = LinearRegression()
ridge_model = Ridge(random_state=RANDOM_STATE)
lasso_model = Lasso(random_state=RANDOM_STATE)

# Обучение моделей
lr_model.fit(X_train, y_train)
ridge_model.fit(X_train, y_train)
lasso_model.fit(X_train, y_train)

# Предсказания на тестовой выборке
y_pred_lr = lr_model.predict(X_test)
y_pred_ridge = ridge_model.predict(X_test)
y_pred_lasso = lasso_model.predict(X_test)

# Вычисление R² для каждой модели
r2_lr = r2_score(y_test, y_pred_lr)
r2_ridge = r2_score(y_test, y_pred_ridge)
r2_lasso = r2_score(y_test, y_pred_lasso)

# Запись результатов в DataFrame
results_regression.loc[0] = ['LR', 'task2', r2_lr]
results_regression.loc[1] = ['Ridge', 'task2', r2_ridge]
results_regression.loc[2] = ['Lasso', 'task2', r2_lasso]

# Вывод результатов
print(results_regression)




   model   task        R2
0     LR  task2  0.668483
1  Ridge  task2  0.665961
2  Lasso  task2  0.666869


3. Для Ridge и Lasso подберите коэффициент регуляризации двумя способами 1) GridSearchCV, 2) RidgeCV и LassoCV, в пределах от $10^{-5}$ до $10^5$ (по степеням 10). Посчитайте R2 на тестовой выборке по всем моделям и сравните с предыдущими результатами.

In [6]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import RidgeCV, LassoCV

# Задание диапазона параметров для регуляризации (по степеням 10)
alphas = np.logspace(-5, 5, 100)

# GridSearchCV для Ridge
ridge_grid_search = GridSearchCV(Ridge(random_state=RANDOM_STATE), 
                                 param_grid={'alpha': alphas}, 
                                 cv=5, 
                                 scoring='r2')
ridge_grid_search.fit(X_train, y_train)
r2_ridge_grid_search = r2_score(y_test, ridge_grid_search.predict(X_test))

# RidgeCV
ridge_cv = RidgeCV(alphas=alphas, cv=5)
ridge_cv.fit(X_train, y_train)
r2_ridge_cv = r2_score(y_test, ridge_cv.predict(X_test))

# GridSearchCV для Lasso
lasso_grid_search = GridSearchCV(Lasso(random_state=RANDOM_STATE), 
                                 param_grid={'alpha': alphas}, 
                                 cv=5, 
                                 scoring='r2')
lasso_grid_search.fit(X_train, y_train)
r2_lasso_grid_search = r2_score(y_test, lasso_grid_search.predict(X_test))

# LassoCV
lasso_cv = LassoCV(alphas=alphas, cv=5, random_state=RANDOM_STATE)
lasso_cv.fit(X_train, y_train)
r2_lasso_cv = r2_score(y_test, lasso_cv.predict(X_test))

# Запись результатов в DataFrame
results_regression.loc[3] = ['Ridge_GridSearchCV', 'task3', r2_ridge_grid_search]
results_regression.loc[4] = ['RidgeCV', 'task3', r2_ridge_cv]
results_regression.loc[5] = ['Lasso_GridSearchCV', 'task3', r2_lasso_grid_search]
results_regression.loc[6] = ['LassoCV', 'task3', r2_lasso_cv]

# Вывод результатов
print("Результаты для задания 2 и 3:")
print(results_regression)

# Сравнение с результатами из задания 2
improvement_ridge_grid_search = r2_ridge_grid_search - results_regression.loc[1, 'R2']
improvement_ridge_cv = r2_ridge_cv - results_regression.loc[1, 'R2']
improvement_lasso_grid_search = r2_lasso_grid_search - results_regression.loc[2, 'R2']
improvement_lasso_cv = r2_lasso_cv - results_regression.loc[2, 'R2']

print("\nСравнение с результатами из задания 2:")
print(f"Ridge (GridSearchCV) улучшение R²: {improvement_ridge_grid_search:.4f}")
print(f"Ridge (RidgeCV) улучшение R²: {improvement_ridge_cv:.4f}")
print(f"Lasso (GridSearchCV) улучшение R²: {improvement_lasso_grid_search:.4f}")
print(f"Lasso (LassoCV) улучшение R²: {improvement_lasso_cv:.4f}")


Результаты для задания 2 и 3:
                model   task        R2
0                  LR  task2  0.668483
1               Ridge  task2  0.665961
2               Lasso  task2  0.666869
3  Ridge_GridSearchCV  task3  0.668483
4             RidgeCV  task3  0.668483
5  Lasso_GridSearchCV  task3  0.668483
6             LassoCV  task3  0.668483

Сравнение с результатами из задания 2:
Ridge (GridSearchCV) улучшение R²: 0.0025
Ridge (RidgeCV) улучшение R²: 0.0025
Lasso (GridSearchCV) улучшение R²: 0.0016
Lasso (LassoCV) улучшение R²: 0.0016


4. Проведите масштабирование выборки (используйте Pipeline, StandardScaler, MinMaxScaler), посчитайте R2 для Ridge и Lasso с параметрами по умолчанию и сравните с предыдущими результатами.

In [7]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Pipeline для Ridge с StandardScaler
ridge_standard_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('model', Ridge(random_state=RANDOM_STATE))
])

ridge_standard_pipeline.fit(X_train, y_train)
r2_ridge_standart_scaler = r2_score(y_test, ridge_standard_pipeline.predict(X_test))

# Pipeline для Ridge с MinMaxScaler
ridge_minmax_pipeline = Pipeline([
    ('scaler', MinMaxScaler()),
    ('model', Ridge(random_state=RANDOM_STATE))
])

ridge_minmax_pipeline.fit(X_train, y_train)
r2_ridge_min_max_scaler = r2_score(y_test, ridge_minmax_pipeline.predict(X_test))

# Pipeline для Lasso с StandardScaler
lasso_standard_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('model', Lasso(random_state=RANDOM_STATE))
])

lasso_standard_pipeline.fit(X_train, y_train)
r2_lasso_standart_scaler = r2_score(y_test, lasso_standard_pipeline.predict(X_test))

# Pipeline для Lasso с MinMaxScaler
lasso_minmax_pipeline = Pipeline([
    ('scaler', MinMaxScaler()),
    ('model', Lasso(random_state=RANDOM_STATE))
])

lasso_minmax_pipeline.fit(X_train, y_train)
r2_lasso_min_max_scaler = r2_score(y_test, lasso_minmax_pipeline.predict(X_test))

# Запись результатов в DataFrame
results_regression.loc[7] = ['Ridge_StandardScaler', 'task4', r2_ridge_standart_scaler]
results_regression.loc[8] = ['Ridge_MinMaxScaler', 'task4', r2_ridge_min_max_scaler]
results_regression.loc[9] = ['Lasso_StandardScaler', 'task4', r2_lasso_standart_scaler]
results_regression.loc[10] = ['Lasso_MinMaxScaler', 'task4', r2_lasso_min_max_scaler]

# Вывод всех результатов
print("Результаты моделей из заданий 2, 3 и 4:")
print(results_regression)

# Сравнение результатов
print("\nСравнение R² моделей из задания 4 с моделями из заданий 2 и 3:")

# Сравнение Ridge с StandardScaler и MinMaxScaler и базовыми результатами
print(f"R² Ridge (StandardScaler): {r2_ridge_standart_scaler:.4f} | R² Ridge (задание 2): {results_regression.loc[1, 'R2']:.4f}")
print(f"R² Ridge (MinMaxScaler): {r2_ridge_min_max_scaler:.4f} | R² Ridge (задание 2): {results_regression.loc[1, 'R2']:.4f}")

# Сравнение Lasso с StandardScaler и MinMaxScaler и базовыми результатами
print(f"R² Lasso (StandardScaler): {r2_lasso_standart_scaler:.4f} | R² Lasso (задание 2): {results_regression.loc[2, 'R2']:.4f}")
print(f"R² Lasso (MinMaxScaler): {r2_lasso_min_max_scaler:.4f} | R² Lasso (задание 2): {results_regression.loc[2, 'R2']:.4f}")




Результаты моделей из заданий 2, 3 и 4:
                   model   task        R2
0                     LR  task2  0.668483
1                  Ridge  task2  0.665961
2                  Lasso  task2  0.666869
3     Ridge_GridSearchCV  task3  0.668483
4                RidgeCV  task3  0.668483
5     Lasso_GridSearchCV  task3  0.668483
6                LassoCV  task3  0.668483
7   Ridge_StandardScaler  task4  0.668190
8     Ridge_MinMaxScaler  task4  0.676221
9   Lasso_StandardScaler  task4  0.624045
10    Lasso_MinMaxScaler  task4  0.257392

Сравнение R² моделей из задания 4 с моделями из заданий 2 и 3:
R² Ridge (StandardScaler): 0.6682 | R² Ridge (задание 2): 0.6660
R² Ridge (MinMaxScaler): 0.6762 | R² Ridge (задание 2): 0.6660
R² Lasso (StandardScaler): 0.6240 | R² Lasso (задание 2): 0.6669
R² Lasso (MinMaxScaler): 0.2574 | R² Lasso (задание 2): 0.6669


5. Подберите коэффициент регуляризации для Ridge и Lasso на масштабированных данных, посчитайте R2 и сравните с предыдущими результатами.

In [8]:
# RidgeCV на данных с StandardScaler
ridge_standard_cv_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('model', RidgeCV(alphas=alphas, cv=5))
])

ridge_standard_cv_pipeline.fit(X_train, y_train)
r2_ridge_standart_scaler_cv = r2_score(y_test, ridge_standard_cv_pipeline.predict(X_test))

# RidgeCV на данных с MinMaxScaler
ridge_minmax_cv_pipeline = Pipeline([
    ('scaler', MinMaxScaler()),
    ('model', RidgeCV(alphas=alphas, cv=5))
])

ridge_minmax_cv_pipeline.fit(X_train, y_train)
r2_ridge_min_max_scaler_cv = r2_score(y_test, ridge_minmax_cv_pipeline.predict(X_test))

# LassoCV на данных с StandardScaler
lasso_standard_cv_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('model', LassoCV(alphas=alphas, cv=5, random_state=RANDOM_STATE))
])

lasso_standard_cv_pipeline.fit(X_train, y_train)
r2_lasso_standart_scaler_cv = r2_score(y_test, lasso_standard_cv_pipeline.predict(X_test))

# LassoCV на данных с MinMaxScaler
lasso_minmax_cv_pipeline = Pipeline([
    ('scaler', MinMaxScaler()),
    ('model', LassoCV(alphas=alphas, cv=5, random_state=RANDOM_STATE))
])

lasso_minmax_cv_pipeline.fit(X_train, y_train)
r2_lasso_min_max_scaler_cv = r2_score(y_test, lasso_minmax_cv_pipeline.predict(X_test))

# Запись результатов в DataFrame
results_regression.loc[11] = ['Ridge_StandardScaler_CV', 'task5', r2_ridge_standart_scaler_cv]
results_regression.loc[12] = ['Ridge_MinMaxScaler_CV', 'task5', r2_ridge_min_max_scaler_cv]
results_regression.loc[13] = ['Lasso_StandardScaler_CV', 'task5', r2_lasso_standart_scaler_cv]
results_regression.loc[14] = ['Lasso_MinMaxScaler_CV', 'task5', r2_lasso_min_max_scaler_cv]

# Вывод всех результатов
print("Результаты моделей из заданий 2, 3, 4 и 5:")
print(results_regression)

# Сравнение результатов
print("\nСравнение R² моделей из задания 5 с моделями из заданий 2, 3 и 4:")

# Сравнение RidgeCV на масштабированных данных с результатами из заданий 2, 3 и 4
print(f"R² Ridge (StandardScaler + CV): {r2_ridge_standart_scaler_cv:.4f} | R² Ridge (задание 2): {results_regression.loc[1, 'R2']:.4f}")
print(f"R² Ridge (MinMaxScaler + CV): {r2_ridge_min_max_scaler_cv:.4f} | R² Ridge (задание 2): {results_regression.loc[1, 'R2']:.4f}")

# Сравнение LassoCV на масштабированных данных с результатами из заданий 2, 3 и 4
print(f"R² Lasso (StandardScaler + CV): {r2_lasso_standart_scaler_cv:.4f} | R² Lasso (задание 2): {results_regression.loc[2, 'R2']:.4f}")
print(f"R² Lasso (MinMaxScaler + CV): {r2_lasso_min_max_scaler_cv:.4f} | R² Lasso (задание 2): {results_regression.loc[2, 'R2']:.4f}")




Результаты моделей из заданий 2, 3, 4 и 5:
                      model   task        R2
0                        LR  task2  0.668483
1                     Ridge  task2  0.665961
2                     Lasso  task2  0.666869
3        Ridge_GridSearchCV  task3  0.668483
4                   RidgeCV  task3  0.668483
5        Lasso_GridSearchCV  task3  0.668483
6                   LassoCV  task3  0.668483
7      Ridge_StandardScaler  task4  0.668190
8        Ridge_MinMaxScaler  task4  0.676221
9      Lasso_StandardScaler  task4  0.624045
10       Lasso_MinMaxScaler  task4  0.257392
11  Ridge_StandardScaler_CV  task5  0.667652
12    Ridge_MinMaxScaler_CV  task5  0.671688
13  Lasso_StandardScaler_CV  task5  0.668482
14    Lasso_MinMaxScaler_CV  task5  0.668484

Сравнение R² моделей из задания 5 с моделями из заданий 2, 3 и 4:
R² Ridge (StandardScaler + CV): 0.6677 | R² Ridge (задание 2): 0.6660
R² Ridge (MinMaxScaler + CV): 0.6717 | R² Ridge (задание 2): 0.6660
R² Lasso (StandardScaler + CV): 

6. Добавьте попарные произведения признаков и их квадраты (используйте PolynomialFeatures) на масштабированных признаках, посчитайте R2 для Ridge и Lasso с параметрами по умолчанию и сравните с предыдущими результатами.

In [9]:
from sklearn.preprocessing import PolynomialFeatures

# Создание PolynomialFeatures (попарные произведения признаков и их квадраты)
poly = PolynomialFeatures(degree=2, include_bias=False)

# Pipeline для Ridge с StandardScaler и PolynomialFeatures
ridge_standard_poly_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('poly', poly),
    ('model', Ridge(random_state=RANDOM_STATE))
])

ridge_standard_poly_pipeline.fit(X_train, y_train)
r2_ridge_standart_scaler_poly = r2_score(y_test, ridge_standard_poly_pipeline.predict(X_test))

# Pipeline для Ridge с MinMaxScaler и PolynomialFeatures
ridge_minmax_poly_pipeline = Pipeline([
    ('scaler', MinMaxScaler()),
    ('poly', poly),
    ('model', Ridge(random_state=RANDOM_STATE))
])

ridge_minmax_poly_pipeline.fit(X_train, y_train)
r2_ridge_min_max_scaler_poly = r2_score(y_test, ridge_minmax_poly_pipeline.predict(X_test))

# Pipeline для Lasso с StandardScaler и PolynomialFeatures
lasso_standard_poly_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('poly', poly),
    ('model', Lasso(random_state=RANDOM_STATE))
])

lasso_standard_poly_pipeline.fit(X_train, y_train)
r2_lasso_standart_scaler_poly = r2_score(y_test, lasso_standard_poly_pipeline.predict(X_test))

# Pipeline для Lasso с MinMaxScaler и PolynomialFeatures
lasso_minmax_poly_pipeline = Pipeline([
    ('scaler', MinMaxScaler()),
    ('poly', poly),
    ('model', Lasso(random_state=RANDOM_STATE))
])

lasso_minmax_poly_pipeline.fit(X_train, y_train)
r2_lasso_min_max_scaler_poly = r2_score(y_test, lasso_minmax_poly_pipeline.predict(X_test))

# Запись результатов в DataFrame
results_regression.loc[15] = ['Ridge_StandardScaler_Poly', 'task6', r2_ridge_standart_scaler_poly]
results_regression.loc[16] = ['Ridge_MinMaxScaler_Poly', 'task6', r2_ridge_min_max_scaler_poly]
results_regression.loc[17] = ['Lasso_StandardScaler_Poly', 'task6', r2_lasso_standart_scaler_poly]
results_regression.loc[18] = ['Lasso_MinMaxScaler_Poly', 'task6', r2_lasso_min_max_scaler_poly]

# Вывод всех результатов
print("Результаты моделей из заданий 2, 3, 4, 5 и 6:")
print(results_regression)

# Сравнение результатов
print("\nСравнение R² моделей из задания 6 с моделями из заданий 2, 3, 4 и 5:")

# Сравнение Ridge с PolynomialFeatures на данных, прошедших через StandardScaler и MinMaxScaler, с результатами из заданий 2, 3, 4 и 5
print(f"R² Ridge (StandardScaler + Poly): {r2_ridge_standart_scaler_poly:.4f} | R² Ridge (задание 2): {results_regression.loc[1, 'R2']:.4f}")
print(f"R² Ridge (MinMaxScaler + Poly): {r2_ridge_min_max_scaler_poly:.4f} | R² Ridge (задание 2): {results_regression.loc[1, 'R2']:.4f}")

# Сравнение Lasso с PolynomialFeatures на данных, прошедших через StandardScaler и MinMaxScaler, с результатами из заданий 2, 3, 4 и 5
print(f"R² Lasso (StandardScaler + Poly): {r2_lasso_standart_scaler_poly:.4f} | R² Lasso (задание 2): {results_regression.loc[2, 'R2']:.4f}")
print(f"R² Lasso (MinMaxScaler + Poly): {r2_lasso_min_max_scaler_poly:.4f} | R² Lasso (задание 2): {results_regression.loc[2, 'R2']:.4f}")




Результаты моделей из заданий 2, 3, 4, 5 и 6:
                        model   task        R2
0                          LR  task2  0.668483
1                       Ridge  task2  0.665961
2                       Lasso  task2  0.666869
3          Ridge_GridSearchCV  task3  0.668483
4                     RidgeCV  task3  0.668483
5          Lasso_GridSearchCV  task3  0.668483
6                     LassoCV  task3  0.668483
7        Ridge_StandardScaler  task4  0.668190
8          Ridge_MinMaxScaler  task4  0.676221
9        Lasso_StandardScaler  task4  0.624045
10         Lasso_MinMaxScaler  task4  0.257392
11    Ridge_StandardScaler_CV  task5  0.667652
12      Ridge_MinMaxScaler_CV  task5  0.671688
13    Lasso_StandardScaler_CV  task5  0.668482
14      Lasso_MinMaxScaler_CV  task5  0.668484
15  Ridge_StandardScaler_Poly  task6  0.817136
16    Ridge_MinMaxScaler_Poly  task6  0.829886
17  Lasso_StandardScaler_Poly  task6  0.732274
18    Lasso_MinMaxScaler_Poly  task6  0.261126

Сравнение R² 

7. Подберите коэффициент регуляризации для Ridge и Lasso на масштабированных данных, добавив PolynomialFeatures, посчитайте R2 и сравните с предыдущими результатами.

In [10]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, MinMaxScaler
from sklearn.linear_model import RidgeCV, LassoCV
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

# Загрузка данных
data = pd.read_csv('boston.csv')

# Подготовка данных
X = data.drop('MEDV', axis=1)
y = data['MEDV']

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Альтернативные значения коэффициентов регуляризации
alphas = np.logspace(-4, 4, 20)  # Увеличение диапазона для более тщательного поиска

# PolynomialFeatures (попарные произведения признаков и их квадраты)
poly = PolynomialFeatures(degree=2, include_bias=False)

# Pipeline для Ridge с StandardScaler, PolynomialFeatures и RidgeCV
ridge_standard_poly_cv_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('poly', poly),
    ('model', RidgeCV(alphas=alphas, cv=5))
])

ridge_standard_poly_cv_pipeline.fit(X_train, y_train)
r2_ridge_standart_scaler_poly_cv = r2_score(y_test, ridge_standard_poly_cv_pipeline.predict(X_test))

# Pipeline для Ridge с MinMaxScaler, PolynomialFeatures и RidgeCV
ridge_minmax_poly_cv_pipeline = Pipeline([
    ('scaler', MinMaxScaler()),
    ('poly', poly),
    ('model', RidgeCV(alphas=alphas, cv=5))
])

ridge_minmax_poly_cv_pipeline.fit(X_train, y_train)
r2_ridge_min_max_scaler_poly_cv = r2_score(y_test, ridge_minmax_poly_cv_pipeline.predict(X_test))

# Pipeline для Lasso с StandardScaler, PolynomialFeatures и LassoCV
lasso_standard_poly_cv_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('poly', poly),
    ('model', LassoCV(alphas=alphas, cv=5, max_iter=200000, tol=1e-4, random_state=42))
])

lasso_standard_poly_cv_pipeline.fit(X_train, y_train)
r2_lasso_standart_scaler_poly_cv = r2_score(y_test, lasso_standard_poly_cv_pipeline.predict(X_test))

# Pipeline для Lasso с MinMaxScaler, PolynomialFeatures и LassoCV
lasso_minmax_poly_cv_pipeline = Pipeline([
    ('scaler', MinMaxScaler()),
    ('poly', poly),
    ('model', LassoCV(alphas=alphas, cv=5, max_iter=200000, tol=1e-4, random_state=42))
])

lasso_minmax_poly_cv_pipeline.fit(X_train, y_train)
r2_lasso_min_max_scaler_poly_cv = r2_score(y_test, lasso_minmax_poly_cv_pipeline.predict(X_test))

# Запись результатов в DataFrame
results_regression.loc[19] = ['Ridge_StandardScaler_Poly_CV', 'task7', r2_ridge_standart_scaler_poly_cv]
results_regression.loc[20] = ['Ridge_MinMaxScaler_Poly_CV', 'task7', r2_ridge_min_max_scaler_poly_cv]
results_regression.loc[21] = ['Lasso_StandardScaler_Poly_CV', 'task7', r2_lasso_standart_scaler_poly_cv]
results_regression.loc[22] = ['Lasso_MinMaxScaler_Poly_CV', 'task7', r2_lasso_min_max_scaler_poly_cv]

# Вывод всех результатов
print("Результаты моделей из заданий 2, 3, 4, 5, 6 и 7:")
print(results_regression)

# Сравнение результатов
print("\nСравнение R² моделей из задания 7 с моделями из заданий 2, 3, 4, 5 и 6:")

# Сравнение RidgeCV с PolynomialFeatures и масштабированием с результатами из заданий 2, 3, 4, 5 и 6
print(f"R² Ridge (StandardScaler + Poly + CV): {r2_ridge_standart_scaler_poly_cv:.4f} | R² Ridge (задание 2): {results_regression.loc[1, 'R2']:.4f}")
print(f"R² Ridge (MinMaxScaler + Poly + CV): {r2_ridge_min_max_scaler_poly_cv:.4f} | R² Ridge (задание 2): {results_regression.loc[1, 'R2']:.4f}")

# Сравнение LassoCV с PolynomialFeatures и масштабированием с результатами из заданий 2, 3, 4, 5 и 6
print(f"R² Lasso (StandardScaler + Poly + CV): {r2_lasso_standart_scaler_poly_cv:.4f} | R² Lasso (задание 2): {results_regression.loc[2, 'R2']:.4f}")
print(f"R² Lasso (MinMaxScaler + Poly + CV): {r2_lasso_min_max_scaler_poly_cv:.4f} | R² Lasso (задание 2): {results_regression.loc[2, 'R2']:.4f}")


C:\Users\285\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:683: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 6.481217564566123, tolerance: 2.764448730650155
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\285\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:683: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 6.912554424862719, tolerance: 2.710494922600619
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\285\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:683: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 48.65145301711209, tolerance: 2.7297569226006195
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\

Результаты моделей из заданий 2, 3, 4, 5, 6 и 7:
                           model   task        R2
0                             LR  task2  0.668483
1                          Ridge  task2  0.665961
2                          Lasso  task2  0.666869
3             Ridge_GridSearchCV  task3  0.668483
4                        RidgeCV  task3  0.668483
5             Lasso_GridSearchCV  task3  0.668483
6                        LassoCV  task3  0.668483
7           Ridge_StandardScaler  task4  0.668190
8             Ridge_MinMaxScaler  task4  0.676221
9           Lasso_StandardScaler  task4  0.624045
10            Lasso_MinMaxScaler  task4  0.257392
11       Ridge_StandardScaler_CV  task5  0.667652
12         Ridge_MinMaxScaler_CV  task5  0.671688
13       Lasso_StandardScaler_CV  task5  0.668482
14         Lasso_MinMaxScaler_CV  task5  0.668484
15     Ridge_StandardScaler_Poly  task6  0.817136
16       Ridge_MinMaxScaler_Poly  task6  0.829886
17     Lasso_StandardScaler_Poly  task6  0.732274
1

8. Подберите наилучшую модель (используйте Pipeline, GridSearchSCV) подбирая тип регуляризации (L1,L2), коэффициент регуляризации, метод масштабирования и степень полинома в PolynomialFeatures. Выведите итоговые параметры и результат R2.

In [11]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, MinMaxScaler
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import r2_score

# Загрузка данных
data = pd.read_csv('boston.csv')

# Подготовка данных
X = data.drop('MEDV', axis=1)
y = data['MEDV']

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Настройка Pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Масштабирование
    ('poly', PolynomialFeatures()),  # Полиномиальные признаки
    ('model', Ridge())  # Регрессия (пока по умолчанию)
])

# Настройка параметров для GridSearchCV
param_grid = {
    'scaler': [StandardScaler(), MinMaxScaler()],
    'poly__degree': [1, 2],  # Степень полинома (без полинома, квадраты)
    'model': [Ridge(), Lasso()],
    'model__alpha': np.logspace(-4, 4, 20)  # Коэффициент регуляризации
}

# Настройка GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='r2', n_jobs=-1)

# Обучение модели
grid_search.fit(X_train, y_train)

# Лучшие параметры и лучший результат
best_params = grid_search.best_params_
r2_best_model = r2_score(y_test, grid_search.best_estimator_.predict(X_test))

# Запись результата в DataFrame
results_regression.loc[23] = ['Best_Model', 'task8', r2_best_model]

# Вывод результатов
print('Параметры лучшей модели:\n', best_params)
print('R2 лучшей модели: {:.4f}'.format(r2_best_model))

# Вывод всех результатов
print("Результаты всех моделей:")
print(results_regression)




Параметры лучшей модели:
 {'model': Ridge(), 'model__alpha': np.float64(29.763514416313132), 'poly__degree': 2, 'scaler': StandardScaler()}
R2 лучшей модели: 0.8215
Результаты всех моделей:
                           model   task        R2
0                             LR  task2  0.668483
1                          Ridge  task2  0.665961
2                          Lasso  task2  0.666869
3             Ridge_GridSearchCV  task3  0.668483
4                        RidgeCV  task3  0.668483
5             Lasso_GridSearchCV  task3  0.668483
6                        LassoCV  task3  0.668483
7           Ridge_StandardScaler  task4  0.668190
8             Ridge_MinMaxScaler  task4  0.676221
9           Lasso_StandardScaler  task4  0.624045
10            Lasso_MinMaxScaler  task4  0.257392
11       Ridge_StandardScaler_CV  task5  0.667652
12         Ridge_MinMaxScaler_CV  task5  0.671688
13       Lasso_StandardScaler_CV  task5  0.668482
14         Lasso_MinMaxScaler_CV  task5  0.668484
15     Rid

In [12]:
results_regression

,model,task,R2
0,LR,task2,0.668483
1,Ridge,task2,0.665961
2,Lasso,task2,0.666869
3,Ridge_GridSearchCV,task3,0.668483
4,RidgeCV,task3,0.668483
5,Lasso_GridSearchCV,task3,0.668483
6,LassoCV,task3,0.668483
7,Ridge_StandardScaler,task4,0.668190
8,Ridge_MinMaxScaler,task4,0.676221
9,Lasso_StandardScaler,task4,0.624045


http://archive.ics.uci.edu/ml/datasets/Adult

In [13]:
data = pd.read_csv('adult.csv')
data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


9. Разделите выборку на признаки и целевую переменную(колонка class). Замените целевую переменную на числовые значения ('<=50K' - 1, '>50K' - 0).

In [14]:
import pandas as pd

# Загрузка данных
data = pd.read_csv('adult.csv')

# Разделение данных на признаки (X) и целевую переменную (y)
X = data.drop('class', axis=1)  # Удаление колонки 'class' из признаков
y = data['class']  # Сохранение колонки 'class' как целевой переменной

# Замена целевой переменной на числовые значения ('<=50K' - 1, '>50K' - 0)
y = y.map({'<=50K': 1, '>50K': 0})

# Проверка результатов
print(y.value_counts())  # Проверка распределения классов
print(X.head())  # Просмотр первых 5 строк признаков

class
1    37155
0    11687
Name: count, dtype: int64
   age         workclass  fnlwgt  education  education-num  \
0   39         State-gov   77516  Bachelors             13   
1   50  Self-emp-not-inc   83311  Bachelors             13   
2   38           Private  215646    HS-grad              9   
3   53           Private  234721       11th              7   
4   28           Private  338409  Bachelors             13   

       marital-status         occupation   relationship   race     sex  \
0       Never-married       Adm-clerical  Not-in-family  White    Male   
1  Married-civ-spouse    Exec-managerial        Husband  White    Male   
2            Divorced  Handlers-cleaners  Not-in-family  White    Male   
3  Married-civ-spouse  Handlers-cleaners        Husband  Black    Male   
4  Married-civ-spouse     Prof-specialty           Wife  Black  Female   

   capital-gain  capital-loss  hours-per-week native-country  
0          2174             0              40  United-States  
1 

10. Посчитайте метрики accuracy и f1_score на предсказании только самого частого класса в целевой переменной.

In [15]:
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score

# Загрузка данных
data = pd.read_csv('adult.csv')

# Разделение данных на признаки (X) и целевую переменную (y)
X = data.drop('class', axis=1)
y = data['class'].map({'<=50K': 1, '>50K': 0})  # Преобразование целевой переменной

# Определение самого частого класса
most_frequent_class = y.mode()[0]

# Предсказания с использованием самого частого класса
y_pred_most_frequent = [most_frequent_class] * len(y)

# Расчет метрик
acc_most_frequent = accuracy_score(y, y_pred_most_frequent)
f1_most_frequent = f1_score(y, y_pred_most_frequent)

# Запись результата в DataFrame
results_classification = pd.DataFrame(columns=['model', 'task', 'f1', 'accuracy'])
results_classification.loc[0] = ['Most Frequent class', 'task10', f1_most_frequent, acc_most_frequent]

# Вывод результатов
print(results_classification)




                 model    task      f1  accuracy
0  Most Frequent class  task10  0.8641  0.760718


11. Выясните, присутствуют ли в данных пропуски. Если присутствуют, заполните их самыми частыми значениями (испольуйте SimpleImputer)

In [16]:
import pandas as pd
from sklearn.impute import SimpleImputer

# Загрузка данных
data = pd.read_csv('adult.csv')

# Проверка наличия пропусков
missing_values = data.isnull().sum()
print("Пропуски в данных:\n", missing_values)

# Если присутствуют пропуски, заполняем их самыми частыми значениями
imputer = SimpleImputer(strategy='most_frequent')
data_imputed = pd.DataFrame(imputer.fit_transform(data), columns=data.columns)

# Проверка на наличие пропусков после заполнения
missing_values_after_imputation = data_imputed.isnull().sum()
print("Пропуски после заполнения:\n", missing_values_after_imputation)




Пропуски в данных:
 age               0
workclass         0
fnlwgt            0
education         0
education-num     0
marital-status    0
occupation        0
relationship      0
race              0
sex               0
capital-gain      0
capital-loss      0
hours-per-week    0
native-country    0
class             0
dtype: int64
Пропуски после заполнения:
 age               0
workclass         0
fnlwgt            0
education         0
education-num     0
marital-status    0
occupation        0
relationship      0
race              0
sex               0
capital-gain      0
capital-loss      0
hours-per-week    0
native-country    0
class             0
dtype: int64


12. Выберите колонки с числовыми и категориальными переменными (используя возможности pandas).

In [17]:
import pandas as pd

# Загрузка данных
data = pd.read_csv('adult.csv')

# Выбор числовых колонок
numeric_columns = data.select_dtypes(include=['number']).columns

# Выбор категориальных колонок
categorical_columns = data.select_dtypes(include=['object']).columns

# Вывод списков числовых и категориальных колонок
print("Числовые колонки:\n", numeric_columns)
print("\nКатегориальные колонки:\n", categorical_columns)



Числовые колонки:
 Index(['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss',
       'hours-per-week'],
      dtype='object')

Категориальные колонки:
 Index(['workclass', 'education', 'marital-status', 'occupation',
       'relationship', 'race', 'sex', 'native-country', 'class'],
      dtype='object')


13. Создайте пайплайн по обработке числовых и категориальных значений колонок (используйте OneHotEncoder,MinMaxScaler) и посчитайте cross_val_score по алгоритмам LogisticRegression, KNeighborsClassifier, LinearSVC по метрикам accuracy и f1_score.

In [19]:
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import make_scorer, f1_score, accuracy_score

# Загрузка данных
data = pd.read_csv('adult.csv')

# Разделение данных на признаки (X) и целевую переменную (y)
X = data.drop('class', axis=1)
y = data['class'].map({'<=50K': 1, '>50K': 0})

# Выбор числовых и категориальных колонок
numeric_columns = X.select_dtypes(include=['number']).columns
categorical_columns = X.select_dtypes(include=['object']).columns

# Создание пайплайна для числовых данных
numeric_transformer = Pipeline(steps=[
    ('scaler', MinMaxScaler())
])

# Создание пайплайна для категориальных данных
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Объединение всех трансформеров в ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_columns),
        ('cat', categorical_transformer, categorical_columns)
    ])

# Создание моделей
models = {
    'LogisticRegression': LogisticRegression(max_iter=1000),
    'KNeighborsClassifier': KNeighborsClassifier(),
    'LinearSVC': LinearSVC(max_iter=10000)
}

# Подсчет cross_val_score для каждой модели по метрикам accuracy и f1_score
results_classification = pd.DataFrame(columns=['model', 'task', 'f1', 'accuracy'])

for i, (model_name, model) in enumerate(models.items()):
    # Создание общего пайплайна
    clf = Pipeline(steps=[('preprocessor', preprocessor), ('classifier', model)])
    
    # Подсчет метрик через cross_val_score
    acc_scores = cross_val_score(clf, X, y, cv=5, scoring=make_scorer(accuracy_score))
    f1_scores = cross_val_score(clf, X, y, cv=5, scoring=make_scorer(f1_score))
    
    # Запись среднего значения метрик в results_classification
    results_classification.loc[i + 1] = [model_name, 'task13', f1_scores.mean(), acc_scores.mean()]

# Вывод результатов
print(results_classification)




                  model    task        f1  accuracy
1    LogisticRegression  task13  0.904781  0.850825
2  KNeighborsClassifier  task13  0.886981  0.824761
3             LinearSVC  task13  0.906294  0.852873


14. Можно заметить что в данных присутствуют значения '?', замените их самыми частыми значениями, (испольуйте SimpleImputer). Посчитайте cross_val_score по алгоритмам LogisticRegression, KNeighborsClassifier, LinearSVC по метрикам accuracy и f1_score.

In [20]:
import numpy as np
import pandas as pd

# Преобразование всех значений pandas.NA в np.nan
X = X.applymap(lambda x: np.nan if pd.isna(x) else x)

# Подготовка и обучение модели
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, f1_score, make_scorer
from sklearn.linear_model import LogisticRegression

# Определение трансформеров для числовых и категориальных данных
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object', 'category']).columns

# Пайплайн для числовых данных
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())])

# Пайплайн для категориальных данных
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# Объединение трансформеров в ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# Обучение модели
model = LogisticRegression()

# Объединение препроцессора и модели в один пайплайн
clf = Pipeline(steps=[('preprocessor', preprocessor), ('classifier', model)])

# Подсчет метрик через cross_val_score
acc_scores = cross_val_score(clf, X, y, cv=5, scoring=make_scorer(accuracy_score))
f1_scores = cross_val_score(clf, X, y, cv=5, scoring=make_scorer(f1_score))

# Запись среднего значения метрик
results_classification['LogisticRegression'] = {
    'Accuracy': np.mean(acc_scores),
    'F1 Score': np.mean(f1_scores)
}
# Вывод результатов
print(results_classification)


C:\Users\285\AppData\Local\Temp\ipykernel_9576\3614646836.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  X = X.applymap(lambda x: np.nan if pd.isna(x) else x)


                  model    task        f1  accuracy  LogisticRegression
1    LogisticRegression  task13  0.904781  0.850825                 NaN
2  KNeighborsClassifier  task13  0.886981  0.824761                 NaN
3             LinearSVC  task13  0.906294  0.852873                 NaN


15. Посчитайте cross_val_score по тем же алгоритмам и метрикам, если просто удалить значения '?'.

In [21]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, f1_score, make_scorer
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC

# Удаление строк с пропущенными значениями '?'
X = X.replace('?', np.nan)
X = X.dropna()

# Обновление целевой переменной y, если были удалены строки из X
y = y.loc[X.index]

# Определение трансформеров для числовых и категориальных данных
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object', 'category']).columns

# Пайплайн для числовых данных
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())])

# Пайплайн для категориальных данных
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# Объединение трансформеров в ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# Функция для подсчета метрик cross_val_score
def calculate_metrics(model):
    clf = Pipeline(steps=[('preprocessor', preprocessor), ('classifier', model)])
    acc_scores = cross_val_score(clf, X, y, cv=5, scoring=make_scorer(accuracy_score))
    f1_scores = cross_val_score(clf, X, y, cv=5, scoring=make_scorer(f1_score))
    return np.mean(f1_scores), np.mean(acc_scores)

# Модели для оценки
models = {
    'LogisticRegression': LogisticRegression(),
    'KNeighborsClassifier': KNeighborsClassifier(),
    'LinearSVC': LinearSVC()
}

# Запись результатов в таблицу
results_classification = pd.DataFrame(columns=['Model', 'Task', 'F1 Score', 'Accuracy'])

for model_name, model in models.items():
    f1_score_mean, acc_score_mean = calculate_metrics(model)
    results_classification.loc[len(results_classification)] = [
        f'{model_name}_delete_missings', 'task15', f1_score_mean, acc_score_mean]

print(results_classification)




                                  Model    Task  F1 Score  Accuracy
0    LogisticRegression_delete_missings  task15  0.902171  0.848569
1  KNeighborsClassifier_delete_missings  task15  0.887012  0.827297
2             LinearSVC_delete_missings  task15  0.902604  0.848835


 16. Посчитайте cross_val_score для RandomForestClassifier,GradientBoostingClassifier на данных с замененными значениями '?' на самые частые значения.

In [22]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, f1_score, make_scorer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

# Замена значений '?' на NaN
X = X.replace('?', np.nan)

# Импутер для замены NaN на самые частые значения
most_frequent_imputer = SimpleImputer(strategy='most_frequent')

# Применение импутора к числовым и категориальным данным
X[numeric_features] = most_frequent_imputer.fit_transform(X[numeric_features])
X[categorical_features] = most_frequent_imputer.fit_transform(X[categorical_features])

# Обновление целевой переменной y, если были удалены строки из X
y = y.loc[X.index]

# Определение трансформеров для числовых и категориальных данных
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# Объединение трансформеров в ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# Функция для подсчета метрик cross_val_score
def calculate_metrics(model):
    clf = Pipeline(steps=[('preprocessor', preprocessor), ('classifier', model)])
    acc_scores = cross_val_score(clf, X, y, cv=5, scoring=make_scorer(accuracy_score))
    f1_scores = cross_val_score(clf, X, y, cv=5, scoring=make_scorer(f1_score))
    return np.mean(f1_scores), np.mean(acc_scores)

# Модели для оценки
models = {
    'RandomForestClassifier': RandomForestClassifier(),
    'GradientBoostingClassifier': GradientBoostingClassifier()
}

# Запись результатов в таблицу
results_classification = pd.DataFrame(columns=['Model', 'Task', 'F1 Score', 'Accuracy'])

for model_name, model in models.items():
    f1_score_mean, acc_score_mean = calculate_metrics(model)
    results_classification.loc[len(results_classification)] = [
        model_name, 'task16', f1_score_mean, acc_score_mean]

print(results_classification)




                        Model    Task  F1 Score  Accuracy
0      RandomForestClassifier  task16  0.901211  0.848215
1  GradientBoostingClassifier  task16  0.912224  0.862899


17. Подберите наилучшую модель, подбирая методы обработки колонок - масштабирование признаков, кодирование признаков и заполнение пропусков. Параметры алгоритмов оставьте по умолчанию. Выведите итоговые параметры и результат accuracy и f1_score.

In [24]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score, make_scorer

# Подготовка данных
X = X.replace('?', np.nan)  # Заменяем '?' на np.nan

# Определение признаков
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = X.select_dtypes(include=['object']).columns.tolist()

# Определение пайплайнов для числовых и категориальных данных
numeric_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Определение ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_pipeline, numeric_features),
        ('cat', categorical_pipeline, categorical_features)
    ]
)

# Определение моделей
models = {
    'RandomForestClassifier': RandomForestClassifier(),
    'GradientBoostingClassifier': GradientBoostingClassifier()
}

# Определение параметров для GridSearchCV
param_grid = {
    'preprocessor__num__imputer__strategy': ['mean', 'median', 'most_frequent'],
    'preprocessor__num__scaler': [StandardScaler(), MinMaxScaler()],
    'preprocessor__cat__imputer__strategy': ['most_frequent', 'constant'],
    'preprocessor__cat__onehot__handle_unknown': ['ignore', 'error']
}

# Инициализация переменных для хранения лучших результатов
best_params = {}
f1_best = 0
acc_best = 0

# Инициализация DataFrame для результатов
results_classification = pd.DataFrame(columns=['Model', 'Task', 'F1 Score', 'Accuracy'])

# Подбор модели и параметров
for model_name, model in models.items():
    pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('classifier', model)])
    
    grid_search = GridSearchCV(
        pipeline,
        param_grid,
        cv=5,
        scoring={'accuracy': make_scorer(accuracy_score), 'f1': make_scorer(f1_score)},
        refit='f1',  # Используем f1 для выбора лучшей модели
        n_jobs=-1
    )
    
    grid_search.fit(X, y)
    
    best_params[model_name] = grid_search.best_params_
    f1_score_mean = grid_search.best_score_
    acc_score_mean = cross_val_score(grid_search.best_estimator_, X, y, cv=5, scoring='accuracy').mean()
    
    if f1_score_mean > f1_best:
        f1_best = f1_score_mean
        acc_best = acc_score_mean
        results_classification = pd.concat([results_classification, pd.DataFrame({
            'Model': [model_name],
            'Task': ['task17'],
            'F1 Score': [f1_best],
            'Accuracy': [acc_best]
        })], ignore_index=True)

print('Параметры лучшей модели:\n', best_params)
print(results_classification)


C:\Users\285\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_search.py:1102: UserWarning: One or more of the test scores are non-finite: [0.84810492 0.84710987 0.84898944 0.84768484 0.84921058 0.84790591
        nan        nan        nan        nan        nan        nan
 0.84799437 0.84887889 0.84797227 0.8483482  0.84848088 0.84823761
        nan        nan        nan        nan        nan        nan]
  warnings.warn(
C:\Users\285\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_search.py:1102: UserWarning: One or more of the test scores are non-finite: [0.90102137 0.90045237 0.90163053 0.90082046 0.90176079 0.90091413
        nan        nan        nan        nan        nan        nan
 0.90092    0.90158006 0.90093911 0.9012535  0.90129917 0.90117761
        nan        nan        nan        nan        nan        nan]
  warnings.warn(
C:\Users\285\AppData\Local\Temp\ipykernel_9576\2390753215.py:81: FutureWarning: The 

Параметры лучшей модели:
 {'RandomForestClassifier': {'preprocessor__cat__imputer__strategy': 'most_frequent', 'preprocessor__cat__onehot__handle_unknown': 'ignore', 'preprocessor__num__imputer__strategy': 'most_frequent', 'preprocessor__num__scaler': StandardScaler()}, 'GradientBoostingClassifier': {'preprocessor__cat__imputer__strategy': 'most_frequent', 'preprocessor__cat__onehot__handle_unknown': 'ignore', 'preprocessor__num__imputer__strategy': 'median', 'preprocessor__num__scaler': StandardScaler()}}
                        Model    Task  F1 Score  Accuracy
0      RandomForestClassifier  task17  0.901761  0.849299
1  GradientBoostingClassifier  task17  0.912252  0.862943
